# Load graph data

In [21]:
import networkx as nx
import json
import numpy as np
import gmplot #for plotting to google maps

#load the shapefile generated from an OSMnx pull (see v4.1 OSMnx to generate SHP.ipynb)
#g = nx.read_shp('data/from_osmnx/edges/edges.shp')
#g = nx.read_shp('data/from_osmnx_2017_05_11/edges/edges.shp')
#g = nx.read_shp('data/from_osmnx_2017_05_20/edges/edges.shp')
g = nx.read_shp('data/from_osmnx_2017_05_21_all_private/edges/edges.shp')

#returns array of coordinates for any edge in the graph
def get_path(n0, n1):
    """If n0 and n1 are connected nodes in the graph, this function
    return an array of point coordinates along the road linking
    these two nodes."""
    return np.array(json.loads(g[n0][n1]['Json'])['coordinates'])

#returns in KILOMETERS
EARTH_R = 6372.8
def geocalc(lat0, lon0, lat1, lon1):
    """Return the distance (in km) between two points in 
    geographical coordinates."""
    lat0 = np.radians(lat0)
    lon0 = np.radians(lon0)
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    dlon = lon0 - lon1
    y = np.sqrt(
        (np.cos(lat1) * np.sin(dlon)) ** 2
         + (np.cos(lat0) * np.sin(lat1) 
         - np.sin(lat0) * np.cos(lat1) * np.cos(dlon)) ** 2)
    x = np.sin(lat0) * np.sin(lat1) + \
        np.cos(lat0) * np.cos(lat1) * np.cos(dlon)
    c = np.arctan2(y, x)
    return EARTH_R * c

#Now, we define a function computing a path's length.
#This function has been found in StackOverflow. http://stackoverflow.com/questions/8858838/need-help-calculating-geographical-distance

#order of lats, lons was incorrect in tutorial! Corrected here
def get_path_length(path):
    return np.sum(geocalc(path[:-1,1], path[:-1,0], path[1:,1], path[1:,0]))

## Load graph data > Add nodes and connect them

Load the data (a Shapefile dataset) with NetworkX. 
This dataset contains detailed information about all roads in San Francisco County. 
NetworkX's read_shp function returns a graph, where each node is a geographical position, 
and each edge contains information about the road linking the two nodes.

In lieu of Census data, SF Gov has a full, connected graph of streets that can be found here: https://data.sfgov.org/Geographic-Locations-and-Boundaries/Streets-of-San-Francisco/4ugn-hf48

In [22]:
"""
    Shapefile is incomplete, so here we manually create edges known to exist
"""
#given two nodes, build a node between them
#nodes must be in (lon, lat) tuple
def create_edge(a,b):
    g.add_edge(a, b)
    g[a][b]['Json'] = '{ "type": "LineString" ,"coordinates" : [' + str([p for p in a]) + ', ' + str([p for p in b]) + ']}'


#road has a gate at the end of public part
p1 = (-122.38385529999999, 37.7423869)
p2 = (-122.382517, 37.741975)
create_edge(p1, p2) #lon/lat

'''extend edge
    inputs: an edge, and a tuple representing a non-existent node
    outputs: the edge is now extended to a new node (original edge endpoint now a coordinate), that node splits the nearest edge
'''
def extend_edge(my_edge, my_point):
    #for the point, find the edge it's on and split it into two edges, connected at the point
    split_edge(my_point)
    
    #for the edge, find the closest end point and create a new edge to the point
    closest, farthest = orient_edge_endpoints(my_point[::-1], my_edge)
    create_edge(closest, my_point[::-1])
    
'''split an edge with a given point on it
    inputs: the point that is being added to the graph
    output: the edge, previously A-B is now two edges, A-C & C-B
'''
def split_edge(my_point):
    my_edge = find_closest_edge(my_point)
    create_edge(my_edge[0], my_point[::-1])
    create_edge(my_edge[1], my_point[::-1])
    
#given a point, return the closest edge; optionally return its distance
def find_closest_edge(my_point, method='all', return_distance=False):

    min_distance_to_coord_edge = 100

#iterate through edges and record distance to closest point on edge
    for n0, n1 in g.edges_iter():

#use two methods to look for closest road: (1) by line segment using perpendicular distance, (2) by edge's coordinates
#(2) is necessary for curving roads...I think
        if n0 == n1:
            continue
        
        my_distance_perp = distance_from_line(n0[::-1], n1[::-1], my_point)
        my_distance_coord = distance_from_edge_coord(n0,n1,my_point)
        
        #for debugging
        if method == 'perp':
            my_distance = my_distance_perp
        elif method == 'coord':
            my_distance = my_distance_coord
        else:
            my_distance = min(my_distance_perp, my_distance_coord)
        
        if my_distance < min_distance_to_coord_edge:
            min_distance_to_coord_edge = my_distance
            closest_edge = [n0, n1]
            
    if return_distance:
        return closest_edge, min_distance_to_coord_edge
            
    return closest_edge

#for a given point, find the edge it is closest to
#first, find it's closest point on an infinite line defined by two nodes
#use https://en.wikipedia.org/wiki/Distance_from_a_point_to_a_line#Line_defined_by_two_points
#if that point IS NOT actually between the nodes, take the closest node

def distance_from_line(p,q,x):

#==Part 1===
#p & q make a line; how far is x from this infinite line?

#vertical traverse
    a = p[1] - q[1]

#horizontal traverse
    b = q[0] - p[0]

#ax+by+c = 0
    c = -a * p[0] - b * p[1]

    
    
#==Part 2== does this closest-line-point actually lie between nodes?
#Ax + By + C = 0 ==> y = (-Ax - C)/B

#first, calculate what the closest point is, holding x constant
#second, calculate the distance between that closest point and the actual point
#finally, check whether the closest point actually lies on the line between the edge's endpoints
#if not, use the closest endpoint

    if b == 0:
# vertical line: ==> x = (-C - By) / A ==> x = -C / A
#thus, ignore y and only take the distance between x and the latitude of our point
    #print('vertical line')
        dist =  abs(a*x[0] + c) / abs(a)
        closest_y = x[1]

# horizontal line: ==> y = (-Ax - C) / B ==> y = -C / B
#thus, ignore x and only take the distance between y and longitude of our point
    elif a == 0:
    #print('horizontal line')
        dist = abs(b* x[1] + c) / abs(b)
        closest_y = -c / b
    

#not a vertical or horizontal line? proceed with full formula
    elif a * b != 0:
#distance = |Ax0 + By0 + C| / sqrt(a^2 + b^s)
    #print('diagnoal line')
        dist = abs(a*x[0] + b*x[1] + c) / (a**2 + b**2)**0.5
        closest_y = (-1 * a * x[0] - c) / b
    
    else:
        print('error!')
        return

#lat&lon must be constrained by nodes' lat/lons
    p_x = distance_points(p, [x[0], closest_y])
    q_x = distance_points(q, [x[0], closest_y])
    p_q = distance_points(p,q)

    error = 1e-10
    if(abs(p_x + q_x - p_q) < error):
        return dist
    
#print('closest point not on edge')
    return min(distance_points(p,x), distance_points(q,x))

#given two 2-d points, return their distance
def distance_points(a,b):
    return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def distance_from_edge_coord(a,b,my_point):
    path = get_path(a,b)
    my_distances = np.sum((path[:,::-1] - my_point)**2, axis=1)
    return min(my_distances**0.5)

'''orient edge endpoints
    inputs: a node and an edge, in any orientation
    outputs: two nodes (the edge's endpoints) in order of closest and farthest
'''
def orient_edge_endpoints(my_node, my_edge):
    distance_0 = distance_points(my_node, my_edge[0])
    distance_1 = distance_points(my_node, my_edge[1])
    
    #if a tie, arbitrarily take the first point
    if distance_0 <= distance_1:
        return my_edge[0], my_edge[1]
    if distance_1 < distance_0:
        return my_edge[1], my_edge[0]
    
this_point = (37.7999762, -122.413069)
this_edge = [(-122.4124276, 37.7997612), (-122.41301, 37.799689)]
#not needed with openstreetmap edits
#extend_edge(this_edge, this_point)

#being directional means being unable to find edges unless in correct order
#so switch to unidirected
g = g.to_undirected()

sg_all = list(nx.connected_component_subgraphs(g.to_undirected()))

#grab the biggest subgraph
max_len = -1
sg_final = sg_all[0]
for sg in sg_all:
    x = len(sg)
    if x > max_len:
        max_len = x
        sg_final = sg
        
# establish all the nodes
nodes = np.array(sg_final.nodes())

#update graph by computing the distance between any two connected nodes
for n0, n1 in sg_final.edges_iter():
    path = get_path(n0, n1)
    distance = get_path_length(path)
    sg_final.edge[n0][n1]['distance'] = distance

## Load graph data > Visually see graph on a map

https://github.com/vgm64/gmplot
library for googplotting

In [23]:
"""
    Plotting the full graph
"""

#for an edge, pull all coordinates
def pull_coords(my_edge):
    return np.array(json.loads(g[e[0]][e[1]]['Json'])['coordinates'])

#plot said coordinates from ALL edges in Shapefile
a_map = gmplot.GoogleMapPlotter.from_geocode("San Francisco")

for e in g.edges_iter():
    a_map.plot(pull_coords(e)[:,1], pull_coords(e)[:,0], 'red', edge_width=2, alpha=0.5)

a_map.draw('test plots/v5 using OSMnx all.html')

# Load run history

In [3]:
from bs4 import BeautifulSoup #library for tcx reading

In [4]:
def find_closest_node(my_point, n=1, distance=False):
    if n == 1:
        return tuple(nodes[np.argmin(np.sum((nodes[:,::-1] - my_point)**2, axis=1))])

    my_closest_nodes = nodes[np.argsort(np.sum((nodes[:,::-1] - my_point)**2, axis=1))[:n]]
    return [tuple(c) for c in my_closest_nodes]


## Load run history > map a KML

In [5]:
file = 'Running SF.kml'

#given two nodes anywhere on the graph, return all necessary intermediary nodes
def connect_nodes(n0, n1):
    return nx.shortest_path(sg_final, 
                        source=n0, 
                        target=n1,
                        weight='distance')

def distance_between_nodes(n0, n1):
    return nx.shortest_path_length(sg_final, source=n0, target=n1, weight='distance')

#given two nodes, add their edge to the map

def plot_edge_gmap(n0, n1, my_map, color='purple', nodes=False):
#check to make sure nodes aren't the same
    if n0 != n1:
        my_path = get_path(n0, n1)
        my_lats = [c[1] for c in my_path]
        my_lons = [c[0] for c in my_path]
        my_map.plot(my_lats, my_lons, color, edge_width=5, alpha=0.4)
        
        if nodes:
            my_map.scatter(my_lats, my_lons, size=5, color=color, marker=False)
            
#given a KML run, plot it on google maps
#inferring closest nodes, edges
def plot_kml(my_coords):

    threshold = 25 / 100000
    run_nodes = []
    
    sf_limit = 1.0
    
    a_map = gmplot.GoogleMapPlotter.from_geocode("San Francisco")
    
    lats = []
    lons = []

#take apart the individual points
    for c in my_coords:
        this_coord = c.split(',')
        lats.append(float(this_coord[1]))
        lons.append(float(this_coord[0]))
        
    for lat, lon in zip(lats, lons):
        this_point = (lat, lon)
    
#if distance is < threshold, then just use the closest node.  Else, find the closest edge.  And plot it 

# need to incorporate coordinates within the edge?

        dist_nodes = np.sum((nodes[:,::-1] - this_point)**2, axis=1)
        dist = min(dist_nodes**0.5)
        
#is the run in SF?
        if dist > sf_limit:
            print('not in SF')
            a_map = gmplot.GoogleMapPlotter(lat, lon, 13)
            #a_map = gmplot.GoogleMapPlotter.from_geocode("San Francisco")
            a_map.scatter(lats, lons, size=15, color='black', marker=False)
            return a_map
            break
    
        if dist < threshold:
            if run_nodes:
#connect the last point to this node, but exclude the last point
                cn = connect_nodes(run_nodes[-1], find_closest_node(this_point))[1:]
            else: #starting condition
                run_nodes.extend([find_closest_node(this_point)])
                continue

            my_color = 'blue'
        else:
            print('guessing edge for ' + str(this_point) + ', distance: ' + str(dist))
            ce = find_closest_edge(this_point, 'all')
            if run_nodes:
                #order matters here, we want to connect nodes that are next to each other, rather than make a long journey
                cn = connect_edge(run_nodes[-1], ce)
                    

            else: #starting condition
                cn = ce #just use the edge
            my_color = 'red'

#take the added nodes and pop them on
        run_nodes.extend(cn)

#how far to start back when plotting
        if run_nodes == cn:
            go_back = 0 - len(cn)
        else:
            go_back = -1 - len(cn)

        
        
        
#to plot: connect the previous node to this one via intermediary nodes, plot coordinates in those edges
        for i in range(go_back, -1):
            plot_edge_gmap(run_nodes[i], run_nodes[i+1], a_map, my_color)
    
        
        a_map.scatter([lat], [lon], size=15, color=my_color, marker=False)
        
    return a_map

#input a messy coord 'lon, lat, 0.0' and return the networkx node (lat, lon)

def coord_to_node(c):
    this_coord = c.split(',')
    lat = float(this_coord[1])
    lon = float(this_coord[0])
    this_point = (lat, lon)
    return this_point

In [15]:
#input must be an existing node (lon, lat) and plot out all the edges from that node
def plot_neighbor_edges(my_point):
#my_point= (-122.38385529999999, 37.7423869)

    a_map = gmplot.GoogleMapPlotter(my_point[1], my_point[0], 14)

    for e in sg_final.neighbors_iter(my_point):
        plot_edge_gmap(my_point, e, nodes=True)
    
    my_map_file = 'v5 - some edges.html'
    a_map.draw(my_map_file)        
    print('mapped to ' + my_map_file)
    
#take a junk point and print out it's closest edges
def plot_closest_edge(my_point, color='purple'):
#this_point = a_run_coords[15]

    my_coord = coord_to_node(my_point)
    my_edge = find_closest_edge(my_coord)

    plot_edge_gmap(my_edge[0], my_edge[1], color)
    
#take a node and an edge, orient the edge's endpoints toward the node and return the path connecting the two
def connect_edge(my_node, my_edge):
    closest, farthest = orient_edge_endpoints(my_node, my_edge)
    
    #if node is on edge, just return the other edge endpoint
    if closest == my_node:
        return [farthest]
    
    my_trio = connect_nodes(my_node, closest)[1:]
    my_trio.extend([farthest])
    return my_trio

#grab kml file
soup = BeautifulSoup(open(file), "html.parser")

#pull a route
test_kmls = [18, 30, 43] +\
list(range(59, 62)) +\
[100, 103, 123, 164, 178, 200, 205, 231, 257, 300, 317, 331, 387] +\
list(range(399, 400)) 

In [7]:
def snap_kml(t):
    a_run = soup.find_all('placemark')[t]
    print(a_run.find('name').get_text())

#pull out lat/lon pairs for each run
    a_run_coords = a_run.coordinates.text.split()

    a_map = plot_kml(a_run_coords)
    location = 'test plots/v5 test_markers - ' + str(t) + '.html'
    a_map.draw(location)
    
    print('plotted to ' + location)

# Plot runs

In [25]:
for t in test_kmls:
    snap_kml(t)

1/29/13
guessing edge for (37.757806, -122.39608500000001), distance: 0.000336786282385
guessing edge for (37.753904, -122.399197), distance: 0.000329560313143
guessing edge for (37.753828, -122.398891), distance: 0.000285063150903
guessing edge for (37.753777, -122.39833799999998), distance: 0.000268601303066
guessing edge for (37.753895, -122.398236), distance: 0.000377132602678
guessing edge for (37.753984, -122.398065), distance: 0.00056473445087
guessing edge for (37.754035, -122.39782300000002), distance: 0.000799270273435
guessing edge for (37.753959, -122.397528), distance: 0.000894652876818
guessing edge for (37.753836, -122.39736700000002), distance: 0.00105933657068
guessing edge for (37.752924, -122.396525), distance: 0.000613117443889
guessing edge for (37.752754, -122.39645), distance: 0.000447457260533
guessing edge for (37.752623, -122.39646600000002), distance: 0.000315520205373
guessing edge for (37.751499, -122.396396), distance: 0.000309965449693
guessing edge for (

In [6]:
import glob
from datetime import datetime #when reading TCX, need to convert dates

#look for all these files, compare it to the KML dates and add it to the map

def pull_from_tcxs(file_pattern, dates, a_map, color):
#loop through all matching files
    for file in glob.glob(file_pattern):

#compare it against dates from KML and only proceed if it is new data 
        file_date = file.split('tapiriik')[1][1:11]
        file_date = datetime.strptime(file_date, '%Y-%m-%d')
        if file_date in dates:
            continue
        print(file_date)

#open it via beautifulsoup
        soup = BeautifulSoup(open(file), "html.parser")

#set up arrays for point data
        lats = []
        lons = []

#not all points contain location data, so loop through 'position' tags and grab the parent
        for pos in soup.find_all('position'):
            tp = pos.parent

#pick out the latitude and longitude (lat/lon need to be floats for plotting)
            this_lat = float(pos.latitudedegrees.string)
            this_lon = float(pos.longitudedegrees.string)

#append trackpoint's data to master lists
            lats.append(this_lat)
            lons.append(this_lon)

#add this route to the plot
        a_map.plot(lats, lons, color, edge_width=4, alpha=0.3)

#given a google map, KML file and color: add to the map for every route manually and record the dates

def pull_from_kml(a_map, file, color):
#initialize list of dates, will later compare to figure which tcx files needed
    dates = []

#grab kml file
    soup = BeautifulSoup(open(file), "html.parser")

#loop through all routes (placemark tags)
    for p in soup.find_all('placemark'):
        #pull date of run
        this_date = p.find('name').text
        if this_date == 'BLOCKED':
            continue
        dates.append(datetime.strptime(this_date, '%m/%d/%y'))

        #pull out lat/lon pairs for each run
        coords = p.coordinates.text.split()

        lats = []
        lons = []

        for c in coords:
            this_coord = c.split(',')
            lats.append(float(this_coord[1]))
            lons.append(float(this_coord[0]))

        a_map.plot(lats, lons, color, edge_width=4, alpha=0.3)

    return dates

#pull from manually built KML
file = 'Running SF.kml'

#define the map area
kml_map = gmplot.GoogleMapPlotter.from_geocode("San Francisco")

#generate plot of all the KML based runs
kml_dates = pull_from_kml(kml_map, file, 'cornflowerblue')

#tcs files are synced from garmin via https://tapiriik.com/
files = '../../../Apps/tapiriik/*.tcx'
pull_from_tcxs(files, kml_dates, kml_map, 'crimson')

#generate all plots
kml_map.draw('runs 2017 05 02.html')
print('all runs mapped')

2014-01-13 00:00:00
2014-01-19 00:00:00
2014-07-15 00:00:00
2014-08-28 00:00:00
2014-12-16 00:00:00
2015-01-10 00:00:00
2015-03-28 00:00:00
2015-07-08 00:00:00
2016-01-06 00:00:00
2016-07-11 00:00:00
2016-08-15 00:00:00
2016-08-15 00:00:00
2016-11-15 00:00:00
2016-11-16 00:00:00
2016-11-24 00:00:00
2016-11-27 00:00:00
2016-11-29 00:00:00
2016-11-30 00:00:00
2016-12-15 00:00:00
2017-01-03 00:00:00
2017-01-08 00:00:00
2017-01-09 00:00:00
2017-01-30 00:00:00
2017-03-05 00:00:00
2017-03-07 00:00:00
2017-03-11 00:00:00
2017-03-11 00:00:00
2017-03-16 00:00:00
2017-03-20 00:00:00
2017-03-22 00:00:00
2017-03-23 00:00:00
2017-03-25 00:00:00
2017-03-26 00:00:00
2017-03-26 00:00:00
2017-03-28 00:00:00
2017-03-31 00:00:00
2017-04-03 00:00:00
2017-04-05 00:00:00
2017-04-08 00:00:00
2017-04-11 00:00:00
2017-04-12 00:00:00
2017-04-13 00:00:00
2017-04-16 00:00:00
2017-04-18 00:00:00
2017-04-20 00:00:00
2017-04-24 00:00:00
2017-04-25 00:00:00
2017-04-26 00:00:00
2017-05-01 00:00:00
2017-05-02 00:00:00


# scratch

In [8]:
import random

In [45]:
random.randrange(0,446)

343

In [46]:
snap_kml(343)

2/22/16
guessing edge for (37.7693239, -122.42164130000002), distance: 0.000594796587075
guessing edge for (37.790591, -122.42825000000002), distance: 0.000572110732302
plotted to test plots/v5 test_markers - 343.html


In [43]:
test_kmls = [18, 30, 43] +\
list(range(59, 62)) +\
[100, 103, 123, 164, 178, 200, 205, 231, 257, 300, 317, 331, 387] +\
list(range(399, 400)) 

In [44]:
test_kmls

[18,
 30,
 43,
 59,
 60,
 61,
 100,
 103,
 123,
 164,
 178,
 200,
 205,
 231,
 257,
 300,
 317,
 331,
 387,
 399]

In [15]:
1.36

In [16]:
a_run

<placemark>
<name>4/17/13</name>
<description><![CDATA[6.59 mi - 53:01<br>8:02 pace<br><br>Flat, windy and sunny!  The park is great, good mix of sun and shade, with wild turkeys all around.  I wouldn't trust the pedestrian crosses at lights, I anticipated one change and almost got creamed when the light just went back to green!]]></description>
<styleurl>#line-6666CC-5</styleurl>
<linestring>
<tessellate>1</tessellate>
<coordinates>-121.28608099999998,38.589747,0.0 -121.286972,38.589487,0.0 -121.287712,38.593646,0.0 -121.28807699999999,38.593638,0.0 -121.28810900000002,38.594032,0.0 -121.288292,38.594929,0.0 -121.28888199999999,38.595868,0.0 -121.289826,38.597109,0.0 -121.289976,38.597319,0.0 -121.290094,38.598275,0.0 -121.291339,38.599977,0.0 -121.291478,38.60053,0.0 -121.291414,38.601302,0.0 -121.29098500000002,38.602149,0.0 -121.28865600000002,38.604328,0.0 -121.29391399999999,38.608194,0.0 -121.29275499999999,38.609401,0.0 -121.293388,38.609761,0.0 -121.29394600000002,38.609954000

In [17]:
len(soup.find_all('placemark'))

446

In [19]:
soup.find_all('placemark')[123]

<placemark>
<name>12/10/13</name>
<description><![CDATA[2.59 mi - 22:37<br>8:43 pace<br><br>Got a bit lost, missed an intended turn at some point along here.  Seems like I'm running the double hill a lot now.]]></description>
<styleurl>#line-6666CC-5</styleurl>
<linestring>
<tessellate>1</tessellate>
<coordinates>-122.428143,37.790574,0.0 -122.417693,37.791965,0.0 -122.417736,37.792338,0.0 -122.418659,37.792202,0.0 -122.417736,37.792338,0.0 -122.41823,37.794525,0.0 -122.411556,37.795441,0.0 -122.4116796,37.7958727,0.0 -122.410011,37.796085,0.0 -122.410183,37.796577,0.0 -122.41823,37.795491,0.0 -122.418401,37.796119,0.0 -122.41926,37.796034,0.0 -122.419174,37.795475,0.0 -122.420011,37.795203,0.0 -122.419817,37.794389,0.0 -122.426405,37.793474,0.0 -122.425933,37.790896,0.0 -122.428143,37.790574,0.0</coordinates>
</linestring>
</placemark>

In [28]:
this_point = a_run_coords[8]
this_point_c = coord_to_node(this_point)

a_map = gmplot.GoogleMapPlotter(this_point_c[0], this_point_c[1], 15)
plot_closest_edge(this_point, color='purple')
a_map.scatter([this_point_c[0]], [this_point_c[1]], color='red', marker=False, size=5)


nears = find_closest_node(this_point_c, n=10, distance=False)
a_map.scatter([n[1] for n in nears], [n[0] for n in nears], color='grey', marker=False, size=4)



my_map_file = 'v5 - closest edge.html'
a_map.draw(my_map_file)

In [25]:
this_point[::-1]

(-122.413069, 37.7999762)

In [46]:
split_edge(this_point)

KeyError: (-122.4159537, 37.7604315)

In [47]:
find_closest_edge(this_point)

KeyError: (-122.4159537, 37.7604315)

In [122]:
distance_from_edge_coord(n0,n1,my_point)

(37.757806, -122.39608500000001)

In [127]:
n0 = correct_edge[0]
n1 = correct_edge[1]


distance_from_line(n0[::-1], n1[::-1], this_point_c)

0.0003367862823853227

In [124]:
n0 = wrong_edge[0]
n1 = wrong_edge[1]


distance_from_line(n0[::-1], n1[::-1], this_point_c)

0.0003367862823853227

In [141]:
#def distance_from_line(p,q,x):

p=n0[::-1]
q=n1[::-1]
x=this_point_c

#==Part 1===
#p & q make a line; how far is x from this infinite line?

#vertical traverse
a = p[1] - q[1]

#horizontal traverse
b = q[0] - p[0]

#ax+by+c = 0
c = -a * p[0] - b * p[1]



#==Part 2== does this closest-line-point actually lie between nodes?
#Ax + By + C = 0 ==> y = (-Ax - C)/B

#first, calculate what the closest point is, holding x constant
#second, calculate the distance between that closest point and the actual point
#finally, check whether the closest point actually lies on the line between the edge's endpoints
#if not, use the closest endpoint

if b == 0:
# vertical line: ==> x = (-C - By) / A ==> x = -C / A
#thus, ignore y and only take the distance between x and the latitude of our point
    print('vertical line')
    dist =  abs(a*x[0] + c) / abs(a)
    closest_y = x[1]

# horizontal line: ==> y = (-Ax - C) / B ==> y = -C / B
#thus, ignore x and only take the distance between y and longitude of our point
elif a == 0:
    print('horizontal line')
    dist = abs(b* x[1] + c) / abs(b)
    closest_y = -c / b


#not a vertical or horizontal line? proceed with full formula
elif a * b != 0:
#distance = |Ax0 + By0 + C| / sqrt(a^2 + b^s)
    print('diagnoal line')
    dist = abs(a*x[0] + b*x[1] + c) / (a**2 + b**2)**0.5
    closest_y = (-1 * a * x[0] - c) / b

else:
    print('error!')
#    return

#lat&lon must be constrained by nodes' lat/lons
p_x = distance_points(p, [x[0], closest_y])
q_x = distance_points(q, [x[0], closest_y])
p_q = distance_points(p,q)


error = 1e-10
if(abs(p_x + q_x - p_q) < error):
    print('on line')
    print(dist)

#print('closest point not on edge')
print(min(distance_points(p,x), distance_points(q,x)))

diagnoal line
on line
4.1255084769045784e-05
0.000336786282385


In [136]:
p_x + q_x == p_q

False

In [137]:
print(p_x + q_x)
print(p_q)

0.000821522975938
0.000821522975938
